# Notebook 04: Agentic Verification Layer (Layer 4)

This notebook demonstrates the hybrid agentic architecture:
- **Triage Node** (rule-based): Routes changes to appropriate handlers
- **Visual Verification Node** (VLM): Confirms ambiguous image changes
- **Drawing Analysis Node** (VLM + OCR): Interprets engineering drawing diffs
- **Table Narration Node** (LLM): Generates natural language for table changes
- **Cross-Version Summary Node** (LLM): Produces executive narrative

**Note:** This notebook requires either:
- Ollama running locally (CPU mode), OR
- GPU with transformers installed (GPU mode)

If neither is available, the triage logic still works without LLM calls.

In [ ]:
import sys
sys.path.insert(0, '..')

import yaml
from src.models.comparison_ir import (
    ChangeType, TextChange, TableChange, ImageChange, DrawingChange,
    PairwiseDiff,
)

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

agent_config = config.get('agent', {})
print(f'Agent device: {agent_config.get("device", "cpu")}')
print(f'Triage config: {agent_config.get("triage", {})}')

## Triage Logic (Rule-Based, No LLM)

The triage node classifies each change and decides whether it needs LLM verification.

In [ ]:
def triage_change(change, config):
    """Decide how to handle a change: 'passthrough', 'visual_verify', 'drawing_analyze', 'table_narrate'."""
    triage = config.get('agent', {}).get('triage', {})
    skip_threshold = triage.get('skip_high_confidence_threshold', 0.95)
    always_verify_drawings = triage.get('always_verify_drawings', True)
    always_verify_ambiguous = triage.get('always_verify_ambiguous_ssim', True)
    
    if change.change_type == ChangeType.UNCHANGED:
        return 'skip'
    
    if isinstance(change, DrawingChange) and always_verify_drawings:
        return 'drawing_analyze'
    
    if isinstance(change, ImageChange) and change.is_ambiguous and always_verify_ambiguous:
        return 'visual_verify'
    
    if isinstance(change, TableChange) and change.total_cell_changes > 0:
        return 'table_narrate'
    
    if change.confidence >= skip_threshold:
        return 'passthrough'
    
    return 'visual_verify'  # Default: verify uncertain changes

# Create sample changes to triage
sample_changes = [
    TextChange(change_type=ChangeType.MODIFIED, page_old=0, page_new=0,
               element_id='p0_text_0_0', old_text='150 PSI', new_text='200 PSI',
               confidence=0.98),
    ImageChange(change_type=ChangeType.MODIFIED, page_old=1, page_new=1,
                element_id='p1_image_0_0', ssim_score=0.82, phash_distance=8,
                confidence=0.60),
    DrawingChange(change_type=ChangeType.MODIFIED, page_old=2, page_new=2,
                  element_id='p2_drawing_0_0', ssim_score=0.91,
                  contour_diff_ratio=0.08, confidence=0.75),
    TableChange(change_type=ChangeType.MODIFIED, page_old=3, page_new=3,
                element_id='p3_table_0_0', modified_cells=[{'row': 1, 'col': 'Qty'}],
                confidence=0.85),
    TextChange(change_type=ChangeType.UNCHANGED, page_old=4, page_new=4,
               element_id='p4_text_0_0', confidence=1.0),
]

print('Triage Results:')
print(f'{"Element ID":<25} {"Type":<15} {"Route":<20} {"Confidence":<10}')
print('-' * 70)
for change in sample_changes:
    route = triage_change(change, config)
    change_type = type(change).__name__.replace('Change', '')
    print(f'{change.element_id:<25} {change_type:<15} {route:<20} {change.confidence:.2f}')

## LLM Integration (via Ollama)

Demonstrate connecting to a local LLM for verification and narration.
This requires Ollama running locally. Install from: https://ollama.ai

In [ ]:
def check_ollama_available():
    """Check if Ollama is running locally."""
    try:
        import requests
        endpoint = config.get('agent', {}).get('ollama_endpoint', 'http://localhost:11434')
        resp = requests.get(f'{endpoint}/api/tags', timeout=5)
        if resp.status_code == 200:
            models = [m['name'] for m in resp.json().get('models', [])]
            print(f'Ollama is running. Available models: {models}')
            return True
    except Exception as e:
        print(f'Ollama not available: {e}')
    print('Skipping LLM demos. Install Ollama and pull a model to test.')
    return False

ollama_available = check_ollama_available()

In [ ]:
def call_ollama(prompt, model='qwen2.5:1.5b'):
    """Call Ollama for text generation."""
    import requests
    endpoint = config.get('agent', {}).get('ollama_endpoint', 'http://localhost:11434')
    resp = requests.post(f'{endpoint}/api/generate', json={
        'model': model,
        'prompt': prompt,
        'stream': False,
    }, timeout=120)
    return resp.json().get('response', 'No response')

# Table narration example
if ollama_available:
    model_name = config['agent']['models']['cpu']['text_llm']
    prompt = """You are a technical document analyst. Describe the following table changes concisely:

Changes detected in parts table on page 3:
- Row modified: Part V-200 (Ball Valve) quantity changed from 2 to 3
- Row modified: Part V-300 (Check Valve) material changed from CS to SS304
- Row added: Part V-400 (Butterfly Valve), Qty=2, Material=SS316

Provide a brief summary suitable for an engineering change notice."""
    
    print(f'Calling {model_name}...')
    response = call_ollama(prompt, model_name)
    print(f'\nTable Narration:\n{response}')
else:
    print('Ollama not available. In production, the table narration node would generate:')
    print('  "Parts table updated: V-200 quantity increased to 3, V-300 upgraded to SS304,"')
    print('  "and new butterfly valve V-400 added in SS316."')

In [ ]:
# Cross-version summary example
if ollama_available:
    model_name = config['agent']['models']['cpu']['text_llm']
    prompt = """You are a technical document analyst. Generate an executive summary of changes across document versions.

V1 -> V2 (15 changes):
- Pressure rating increased from 150 to 200 PSI (page 1)
- Parts table: V-200 quantity increased, V-300 material upgraded
- Drawing A: bolt pattern modified, 2 new dimensions added

V2 -> V3 (8 changes):
- Added Section 3: Safety procedures
- Parts table: V-400 butterfly valve added
- Drawing A: fillet radius increased from R5 to R8

Provide a 3-4 sentence executive summary for a project review meeting."""

    print(f'Calling {model_name} for cross-version summary...')
    response = call_ollama(prompt, model_name)
    print(f'\nExecutive Summary:\n{response}')
else:
    print('Ollama not available. Summary would be generated by the LLM agent.')

In [ ]:
print('Done! Agentic verification layer validated.')